#### Entrenamiento personalizado StyleGan2-ADA
StyleGAN2-ADA solo funciona con Tensorflow 1. Es necesario ejecutar la siguiente celda antes que todo, para asegurarnos de usar TF1 y no TF2.

Notebook creado por [Derrick Schultz ](https://github.com/dvschultz)y [Lia Coleman](https://github.com/Lia-C). 
Traducido al español por [Laboratorio Hypereikon](https://github.com/hypereikon), Sebastián Rojas y Constanza Lobos.




In [ ]:
#@markdown instalar tensorflow
%tensorflow_version 1.x

#### Instalar repositorio en tu drive

Es complicado entrenar en colab. La mejor manera es instalando directamente el repositorio en una carpeta de google drive.





In [ ]:
#@markdown Primero, montar el drive en el notebook de colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown Ahora, ejecuta esta celda. Si ya instalaste previamente el repositorio, se saltará el proceso de instalación, si no lo haz instalado, instalará todos los archivos necesarios.
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada"):
    %cd "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

%cd "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git checkout origin/main -- train.py

### Convertir dataset a .tfrecords
**Nota: Esto solo hay que ejecutarlo una vez por dataset. Si ya haz ejecutado esta celda y vuelves a continuar entrenando ignora esta celda**  

Necesitamos un dataset de imagenes a 1024x1024px.

Puedes subir tu dataset diractamente a Colab (como un archivo .zip), o puedes subirlo a drive y leerlo desde ahí editando `zip_path`.

In [ ]:
#@markdown copia la ruta del .zip y descomprimelo:
zip_path = "/content/CAT1024.zip"  #@param {type: "string"}

!unzip {zip_path} -d /content/

Ahora que el dataset de imagenes ya está subido, necesitamos convertirlo al formato `.tfrecords`
Dependiendo de la resolución de tus imagenes y cuantas tienes, esto puede tardar un rato. 

In [ ]:
#@markdown actualiza a la ruta de la carpeta con imagenes
dataset_path = "/content/CAT1024"  #@param {type: "string"}
#@markdown ponle un nombre a tu dataset
dataset_name = "cat1024" #@param {type: "string"}

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

## Entrenar un modelo personalizado

Ahora estamos listxs para entrenar! Hay varios argumentos al entrenar, los de abajo son las opciones más utilizadas. Para ver todas las opciones, ejecuta la siguiente celda.

In [ ]:
#@markdown detalle parametros
!python train.py --help

In [ ]:
#@markdown este debe coincidir exactamente con el nombre usado para el archivo .tfrecords
dataset_name = "dante1024" #@param {type: "string"}
#@markdown que tan seguido el modelo debe entregar samples y respaldo del modelo en archivo .pkl
snapshot_count = 4 #@param {type: "number"}
#@markdown deberian las imagenes en la aumentacion reflejar horizontalmente?
mirrored = True #@param {type:"boolean"}
#@markdown deberian las imagenes en la aumentacion reflejar verticalmente?
mirroredY = True #@param {type:"boolean"}
#metricas? 
metric_list = None
#@markdown aumentaciones
augs = "bg" #@param {type: "string"}

#
#@markdown  esta es la linea más importante de actualizar

#@markdown  si estas empezando a entrenar un modelo seteala a ffhq(+resolution)  

#@markdown  si continuas entrenandolo, copia la ruta del del ultimo .pkl  
resume_from = "/content/drive/MyDrive/colab-sg2-ada2/stylegan2-ada/results/00008-dante1024-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000160.pkl" #@param {type: "string"}

#no editas esta a no ser que sepas lo que estás haciendo :)
!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from} --augpipe="bg"

### Mientras este entrenando...
**Una vez esté ejecutandose la celda anterior, deberia estar entrenandose!**

No cierres esta pestaña! Colab necesita estar abierto y corriendo en orden para continuar entrenando. Cada ~15min deberia entregar una nueva linea al output, indicando que sigue entrenando. Dependiendo de como seteaste `snapshot_count` guardará los checkpoints en tu carpeta de google drive con samples (`fakesXXXXXx.jpg`) y los weights del modelo (`network-snapshot-XXXXXX.pkl`). Los samples son interesantes de mirar mientras se entrena, pero no te compliques por cada sample individual.

Si eliges una metrica, además te entregará los scores de cada snapshot. No te obsesiones con estos! Son solo una guia, puedes subir y bajar un poco entre cada snapshot. Lo que deberias esperar es que el score baje gradualmente con el tiempo.

Una vez que Colab se apague, puedes reconectar el notebook y re-ejecutar cada celda desde arriba hasta abajo. Asegurate de actualizar la ruta de `resume_from` para continuar entrenando desde el ultimo modelo.